# Forecasting Optuna Search CV 
## Modulus Set 3

**Notebook Goal**
- A modeling pipeline that optimizes the hyperparameters of the sktime forecasters that have the [capavility:pred_int tag](https://www.sktime.net/en/stable/examples/01b_forecasting_proba.html) 
- This notebook will focus on the ones where `i mod 4 = 3` wher `i` is the index of the registry table in the above link.
- The work will be based on this documentation: [ForecastingOptunaSearchCV](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ForecastingOptunaSearchCV.html)

In [36]:
from sktime.registry import all_estimators
from dotenv import load_dotenv
import sys
import os

# Load environment variables from .env file
load_dotenv()
repo_dir = os.path.abspath(os.path.join(os.getcwd(), "../../"))  # points to the root
src_dir = os.path.join(repo_dir, "src")
sys.path.insert(0, repo_dir)
# Now the import works! :)
from src.data.data_loader import load_data
from src.data.data_cleaner import clean_data

# Load the data
df = load_data()
# Get the list of forecasters with prediction interval capability
models = all_estimators(
    "forecaster", filter_tags={"capability:pred_int": True}, as_dataframe=True
)


c:\Users\viloh\Documents\nocturnal-hypo-gly-prob-forecast\src\data\data_loader.py:32: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, usecols=keep_columns)


In [37]:
clean_data(df) # in place
print(df.columns)

Index(['id', 'p_num', 'time', 'insulin-5:55', 'insulin-5:50', 'insulin-5:45',
       'insulin-5:40', 'insulin-5:35', 'insulin-5:30', 'insulin-5:25',
       ...
       'cals-0:40', 'cals-0:35', 'cals-0:30', 'cals-0:25', 'cals-0:20',
       'cals-0:15', 'cals-0:10', 'cals-0:05', 'cals-0:00', 'bg+1:00'],
      dtype='object', length=364)
